In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
#PROJECT_ROOT = '/content/drive/My Drive/tfm/codigo' #@param {type:"string"}
#os.chdir(PROJECT_ROOT)
#print("Current working directory:", os.getcwd())

Current working directory: /content/drive/My Drive/tfm/codigo


In [4]:
import sys
import torch
import torch
import torch.nn as nn
import numpy as np

import joblib

In [5]:
sys.path.append('/content/drive/My Drive/tfm/codigo/modelo_ann/')  # Add the path to your models
from modelo import MLP

In [ ]:
# === Cargar modelo y escaladores ===
model = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/modelo_ann/modelo_ann.pkl")

scaler_entrada = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/modelo_ann/standardscaler_datos_entrada.pkl")
scaler_precio = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/modelo_ann/standardscaler_precio.pkl")


In [7]:
# === Input manual ===
# El precio_medio_mun_tipo ya debe estar en euros → le aplicas log
# Todas las variables deben coincidir con las columnas usadas para entrenar

input_dict = {
    'Habitaciones': 3,
    'Aseos': 2,
    'Metros': 105,
    'CUDIS': 2408907,
    'Población': 120000,
    'Renta bruta media por persona': 36000,
    'Comodidades': 1,
    'Capital': 1,
    'Precio_medio_mun_tipo': np.log(132000)  # € → aplicar log
}


In [30]:
# === Preprocesado ===

# 1. Define the column order (including 'Capital')
column_order = ['Habitaciones', 'Aseos', 'Metros', 'CUDIS', 'Población',
                'Renta bruta media por persona', 'Comodidades', 'Capital',
                'Precio_medio_mun_tipo']

# 2. Features to be scaled (excluding 'Capital')
features_to_scale = [feat for feat in column_order if feat != 'Capital']

# 3. Create input data for scaling
X_input_to_scale = pd.DataFrame([input_dict], columns=features_to_scale)

# 4. Scale the selected features
X_scaled = scaler_entrada.transform(X_input_to_scale)

# 5. Insert 'Capital' at the penultimate position
capital_index = column_order.index('Capital')
X_with_capital = np.insert(X_scaled, capital_index, input_dict['Capital'], axis=1)
print(X_with_capital)

# 6. Convert to PyTorch tensor
X_tensor = torch.tensor(X_with_capital, dtype=torch.float32)

[[ 0.24348198  0.87927311  0.36340549 -0.44528777  0.51035166  1.05159444
   0.90479803  1.          0.54688789]]


In [23]:
# === Predicción ===
with torch.no_grad():
    y_pred_scaled = model(X_tensor).item()

# === Inversión de escalado + exponencial ===
y_pred_log = scaler_precio.inverse_transform([[y_pred_scaled]])[0][0]
y_pred_eur = np.exp(y_pred_log)

print(f"📦 Precio estimado: {y_pred_eur:,.2f} €")

📦 Precio estimado: 150,779.76 €


In [ ]:
import numpy as np
import pandas as pd
import torch

def predecir_precio(input_dict, scaler_x, scaler_y, model):
    """
    Predice el precio final en euros a partir de un input_dict con características de la vivienda.

    Parámetros:
        input_dict: Diccionario con los valores de entrada (en euros y sin escalar).
        scaler_x: StandardScaler entrenado para X (entrada).
        scaler_y: StandardScaler entrenado para y (salida en log).
        model: Red neuronal entrenada (PyTorch).

    Retorna:
        Precio estimado (en euros).
    """

    # Orden correcto de las columnas (sin 'Capital' si no fue usada para entrenar el modelo)
    column_order = ['Habitaciones', 'Aseos', 'Metros', 'CUDIS',
                    'Población', 'Renta bruta media por persona',
                    'Comodidades', 'Capital', 'Precio_medio_mun_tipo']

    # Asegurar que el precio medio ya está en log en el input_dict
    input_dict['Precio_medio_mun_tipo'] = np.log(input_dict['Precio_medio_mun_tipo'])


    # 2. Features to be scaled (excluding 'Capital')
    features_to_scale = [feat for feat in column_order if feat != 'Capital']

    # 3. Create input data for scaling
    X_input_to_scale = pd.DataFrame([input_dict], columns=features_to_scale)

    # 4. Scale the selected features
    X_scaled = scaler_x.transform(X_input_to_scale)

    # 5. Insert 'Capital' at the penultimate position
    capital_index = column_order.index('Capital')
    X_with_capital = np.insert(X_scaled, capital_index, input_dict['Capital'], axis=1)

    # 6. Convert to PyTorch tensor
    X_tensor = torch.tensor(X_with_capital, dtype=torch.float32)

    # Predecir con el modelo
    model.eval()
    with torch.no_grad():
        y_pred_scaled = model(X_tensor).numpy().ravel()

    # Invertir escalado
    y_pred_log = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

    # Invertir logaritmo para obtener el precio en euros
    y_pred_euros = np.exp(y_pred_log)[0]

    return y_pred_euros

In [29]:
input_dict = {
    'Habitaciones': 3,
    'Aseos': 2,
    'Metros': 105,
    'CUDIS': 2408907,
    'Población': 120000,
    'Renta bruta media por persona': 36000,
    'Comodidades': 1,
    'Capital': 1,
    'Precio_medio_mun_tipo': 132000  # en euros → la función aplica log
}

precio_estimado = predecir_precio(input_dict, scaler_entrada, scaler_precio, model)
print(f"💰 Precio estimado: {precio_estimado:,.2f} €")
print(f"El rango de precios 💰 este inmueble es: {(precio_estimado*0.7):,.2f} € y {(precio_estimado*1.3):,.2f} €")

💰 Precio estimado: 150,779.73 €
El rango de precios 💰 este inmueble es: 105,545.81 € y 196,013.65 €
